In [13]:
import csv
import datetime
import os
import pickle
import subprocess
# Imports above are standard library
# Imports below are 3rd-party
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import RocCurveDisplay, ConfusionMatrixDisplay, recall_score, precision_score, silhouette_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, CategoricalNB, ComplementNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from statsmodels.api import qqplot
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import scipy.stats as stats
import statsmodels.api as sm
from xgboost import XGBClassifier, plot_importance

## Kaggle API
You will need your own API key, which you can get by following instructions at https://github.com/Kaggle/kaggle-api/blob/main/docs/README.md

In [3]:
os.environ["KAGGLE_USERNAME"] = "jsf80238"
os.environ["KAGGLE_KEY"] = "4036359324650e1c13adfc7ba87ff90e"
import kaggle  # The import itself uses your KAGGLE_KEY, that's why the import is not at the top. Really.

dataset_name = "NCHS - Death Rates and Causes of Death"
command = "kaggle datasets list --csv --search".split()
#command.append("NCHS - Death Rates and Causes of Death")
result = subprocess.run(command + [dataset_name], capture_output=True)
data = result.stdout.decode(encoding="utf-8")
data[:100]

'ref,title,size,lastUpdated,downloadCount,voteCount,usabilityRating\r\ncdc/nchs-death-rates-and-causes-'

In [4]:
csvreader = csv.DictReader(data.splitlines())
for row in csvreader:
    if row["title"] == dataset_name:
        print(row)
        dataset_ref = row['ref']
        break
dataset_ref

{'ref': 'cdc/nchs-death-rates-and-causes-of-death', 'title': 'NCHS - Death Rates and Causes of Death', 'size': '3MB', 'lastUpdated': '2019-12-28 01:24:17', 'downloadCount': '1540', 'voteCount': '14', 'usabilityRating': '0.7647059'}


'cdc/nchs-death-rates-and-causes-of-death'

In [5]:
dataset_ref = "cdc/nchs-death-rates-and-causes-of-death"
file_name = "nchs-leading-causes-of-death-united-states.csv"
command = f"kaggle datasets download --unzip --force --file {file_name} {dataset_ref}".split()
result = subprocess.run(command, capture_output=True)
stdout = result.stdout.decode(encoding="utf-8")
stderr = result.stderr.decode(encoding="utf-8")
print(stdout)
print(stderr)
print(os.linesep.join(sorted(os.listdir("."))))



100%|██████████| 834k/834k [00:00<00:00, 3.18MB/s]

.ipynb_checkpoints
death_rate.ipynb
nchs-age-adjusted-death-rates-for-selected-major-causes-of-death.csv
nchs-leading-causes-of-death-united-states.csv
socrata_metadata_nchs-age-adjusted-death-rates-for-selected-major-causes-of-death.json
socrata_metadata_nchs-death-rates-and-life-expectancy-at-birth.json
socrata_metadata_nchs-leading-causes-of-death-united-states.json
socrata_metadata_nchs-potentially-excess-deaths-from-the-five-leading-causes-of-death.json
socrata_metadata_nchs-top-five-leading-causes-of-death-united-states-1990-1950-2000.json


## Kaggle workaround
If you were unable to download the cause-of-death file you can also grab it from my GitHub repo:


In [126]:
df = pd.read_csv(file_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10868 entries, 0 to 10867
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     10868 non-null  int64  
 1   113 Cause Name           10868 non-null  object 
 2   Cause Name               10868 non-null  object 
 3   State                    10868 non-null  object 
 4   Deaths                   10868 non-null  int64  
 5   Age-adjusted Death Rate  10868 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 509.6+ KB


In [127]:
YEAR = "year"
CAUSE = "cause_name"
STATE_NAME = "state_name"
DEATH_RATE = "death_rate"
df.drop(columns=["113 Cause Name", "Deaths"], inplace=True)
name_dict = {
    "Year": YEAR,
    "Cause Name": CAUSE,
    "State": STATE_NAME,
    "Age-adjusted Death Rate": DEATH_RATE,
}
df.rename(columns=name_dict, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10868 entries, 0 to 10867
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        10868 non-null  int64  
 1   cause_name  10868 non-null  object 
 2   state_name  10868 non-null  object 
 3   death_rate  10868 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 339.8+ KB


In [128]:
mask = (df[YEAR] >= 2010) & (df[STATE_NAME] != 'United States')
df = df[mask]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4488 entries, 0 to 10802
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   year        4488 non-null   int64  
 1   cause_name  4488 non-null   object 
 2   state_name  4488 non-null   object 
 3   death_rate  4488 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 175.3+ KB


In [129]:
df = df[[STATE_NAME, CAUSE, DEATH_RATE]].groupby([STATE_NAME, CAUSE]).mean().reset_index()
df

,state_name,cause_name,death_rate
0,Alabama,All causes,924.6125
1,Alabama,Alzheimer's disease,35.2625
2,Alabama,CLRD,55.5750
3,Alabama,Cancer,180.5125
4,Alabama,Diabetes,22.6875
...,...,...,...
556,Wyoming,Influenza and pneumonia,17.5750
557,Wyoming,Kidney disease,11.7625
558,Wyoming,Stroke,33.2625
559,Wyoming,Suicide,24.6750


## Census data
Nonemployer Statistics provides annual statistics on U.S. businesses with no paid employees or payroll at a detailed geography and industry level.

Statistics are available on businesses that have no paid employment or payroll, are subject to federal income taxes, and have receipts of $1,000 or more ($1 or more for the Construction sector). The data are available for approximately 450 NAICS industries at the national, state, county, metropolitan statistical area, and combined statistical area geography levels. The majority of NAICS industries are included.

https://www.census.gov/data/developers/data-sets/nonemp-api.2017.html#list-tab-1358655114

https://www.census.gov/data/developers/data-sets/nonemp-api.html
https://api.census.gov/data/2017/nonemp/variables.html

https://www2.census.gov/programs-surveys/nonemployer-statistics/technical-documentation/record-layouts/state-record-layout/state_record_layout_2017.txt

### Definitions
* LFO = Legal form of organization
* NAICS2017_LABEL = type of business
* NAME = geo areas, including states
* NESTAB = Number of nonemployer establishments
* NRCPTOT = Nonemployer sales, value of shipments, or revenue ($1,000)
* RCPSZES_LABEL = Separates establishments into groups by sales/receipts, including "All establishments"

In [130]:
CENSUS_API_KEY = "35b2edd7f868b6d6f79e5988091d0f8df6ffbd2a"

In [131]:
STATE_KEY = "state_key"
POPULATION = "population"
KEY = "key"
CENSUS_DEMOGRAPHICS_URL = "https://api.census.gov/data/2019/pep/charagegroups"
param_dict = {'get': 'NAME,POP', 'for': 'state:*', KEY: CENSUS_API_KEY}

In [132]:
response = requests.get(CENSUS_DEMOGRAPHICS_URL, params=param_dict)

In [133]:
result_list = response.json()  # The return from the API call is a list of lists, with first item being the column names
column_name_list = result_list.pop(0)
states_df = pd.DataFrame(result_list, columns=column_name_list)
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   NAME    52 non-null     object
 1   POP     52 non-null     object
 2   state   52 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [134]:
name_dict = {
    "NAME": STATE_NAME,
    "POP": POPULATION,
    "state": STATE_KEY,
}
states_df.rename(columns=name_dict, inplace=True)
states_df[POPULATION] = states_df[POPULATION].astype(int)
states_df[STATE_KEY] = states_df[STATE_KEY].astype(int)
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   state_name  52 non-null     object
 1   population  52 non-null     int64 
 2   state_key   52 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB


In [135]:
states_df[POPULATION].sum()  # Expected value ~330 million

331433217

In [136]:
df = df.merge(states_df, on=STATE_NAME, how="inner")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   state_name  561 non-null    object 
 1   cause_name  561 non-null    object 
 2   death_rate  561 non-null    float64
 3   population  561 non-null    int64  
 4   state_key   561 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 22.0+ KB


In [137]:
NESTAB = "NESTAB"
NRCPTOT = "NRCPTOT"
LFO = "LFO"
NAICS2017 = "NAICS2017"

CENSUS_NONEMPLOYER_URL = "https://api.census.gov/data/2017/nonemp"
param_dict = {'get': ','.join((LFO, NESTAB, NRCPTOT)), 'for': 'state:*', NAICS2017: '*', KEY: CENSUS_API_KEY}

In [138]:
# This cell may take a minute or two
response = requests.get(CENSUS_NONEMPLOYER_URL, params=param_dict)

In [139]:
result_list = response.json()  # The return from the API call is a list of lists, with first item being the column names
column_name_list = result_list.pop(0)
business_df = pd.DataFrame(result_list, columns=column_name_list)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109381 entries, 0 to 109380
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   LFO        109381 non-null  object
 1   NESTAB     109381 non-null  object
 2   NRCPTOT    109381 non-null  object
 3   NAICS2017  109381 non-null  object
 4   state      109381 non-null  object
dtypes: object(5)
memory usage: 4.2+ MB


## Census codes
I've gone to the trouble of downloading these.
* North American Industry Classification System (aka NAIC)
  * [Original](https://www.census.gov/naics/?58967?yearbck=2017)
  * [Used version](https://github.com/jsf80238/data_science/blob/main/data_files/naics_codes.csv)
* Legal Form of Organization
  * [Original](https://www2.census.gov/programs-surveys/abs/technical-documentation/api/NESD-Technical-Employer-and-Nonemployer-API-5242022Final.pdf)
  * [Used version](https://github.com/jsf80238/data_science/blob/main/data_files/legal_form_codes.csv)

In [140]:
LEGAL_FORM = "legal_form"
BUSINESS_TYPE = "business_type"
ESTABLISHMENT_COUNT = "establishment_count"
REVENUE_IN_THOUSANDS = "revenue_in_thousands"
REVENUE_CATEGORY = "revenue_category"
name_dict = {
    "state": STATE_KEY,
    "NAICS2017": BUSINESS_TYPE,
    "LFO": LEGAL_FORM,
    "NESTAB": ESTABLISHMENT_COUNT,
    "NRCPTOT": REVENUE_IN_THOUSANDS,
}
business_df.rename(columns=name_dict, inplace=True)
business_df = business_df.reindex([STATE_KEY, LEGAL_FORM, BUSINESS_TYPE, ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS], axis=1)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109381 entries, 0 to 109380
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   state_key             109381 non-null  object
 1   legal_form            109381 non-null  object
 2   business_type         109381 non-null  object
 3   establishment_count   109381 non-null  object
 4   revenue_in_thousands  109381 non-null  object
dtypes: object(5)
memory usage: 4.2+ MB


In [141]:
for column_name in ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS, STATE_KEY:
    business_df[column_name] = business_df[column_name].astype(int)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109381 entries, 0 to 109380
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   state_key             109381 non-null  int64 
 1   legal_form            109381 non-null  object
 2   business_type         109381 non-null  object
 3   establishment_count   109381 non-null  int64 
 4   revenue_in_thousands  109381 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.2+ MB


In [142]:
business_df

,state_key,legal_form,business_type,establishment_count,revenue_in_thousands
0,28,001,00,219596,8889009
1,28,9101,00,1750,206357
2,28,9111,00,6527,848178
3,28,920,00,198380,6220549
4,28,930,00,12939,1613925
...,...,...,...,...,...
109376,26,001,813,5972,86303
109377,26,9101,813,135,13364
109378,26,9111,813,19,2159
109379,26,920,813,5803,69991


## Check for duplicate rows

In [143]:
mask = business_df[[STATE_KEY, BUSINESS_TYPE, LEGAL_FORM]].duplicated()
business_df[mask]

,state_key,legal_form,business_type,establishment_count,revenue_in_thousands


## Check for holes
Verify this data does not have holes, meaning verify every combination of legal_form/business_type exists for every state.

If something is missing, add it with zero for the establishment_count and revenue_in_thousands.

In [144]:
from itertools import product
state_list = business_df[STATE_KEY].unique()
legal_form_list = business_df[LEGAL_FORM].unique()
business_type_list = business_df[BUSINESS_TYPE].unique()
wanted_combinations_set = set(product(state_list, legal_form_list, business_type_list))
print(f"{len(wanted_combinations_set):,} possible combinations, for example:")
for i, item in enumerate(wanted_combinations_set):
    print(item)
    if i > 10:
        break


119,850 possible combinations, for example:
(19, '9101', '511')
(12, '9101', '621111')
(20, '920', '237')
(29, '9101', '4241')
(44, '930', '6216')
(23, '001', '3241')
(33, '9111', '56141')
(18, '9101', '54136')
(10, '930', '445')
(19, '920', '54134')
(50, '9101', '722511')
(33, '001', '339')


In [145]:
existing_combinations_set = set()
for item in business_df[[STATE_KEY, LEGAL_FORM, BUSINESS_TYPE]].to_records(index=False):
    existing_combinations_set.add(tuple(item))
print(f"{len(existing_combinations_set):,} existing combinations, for example:")
for i, item in enumerate(existing_combinations_set):
    print(item)
    if i > 10:
        break

109,381 existing combinations, for example:
(19, '9101', '511')
(12, '9101', '621111')
(20, '920', '237')
(29, '9101', '4241')
(23, '001', '3241')
(10, '930', '445')
(19, '920', '54134')
(50, '9101', '722511')
(33, '001', '339')
(22, '920', '53228')
(25, '001', '4247')
(31, '930', '81222')


In [146]:
missing_combinations_set = wanted_combinations_set - existing_combinations_set
print(f"{len(missing_combinations_set):,} missing combinations, for example:")
for i, item in enumerate(missing_combinations_set):
    print(item)
    if i > 10:
        break

10,469 missing combinations, for example:
(15, '930', '532281')
(21, '930', '621391')
(15, '9101', '21311')
(44, '9101', '331')
(44, '930', '6216')
(38, '9101', '45114')
(33, '9111', '56141')
(15, '9101', '1151')
(18, '9101', '54136')
(41, '9111', '81293')
(35, '920', '53311')
(22, '9101', '44613')


In [147]:
missing_data = [list(combo) + [0, 0] for combo in missing_combinations_set]
missing_data[:10]

[[15, '930', '532281', 0, 0],
 [21, '930', '621391', 0, 0],
 [15, '9101', '21311', 0, 0],
 [44, '9101', '331', 0, 0],
 [44, '930', '6216', 0, 0],
 [38, '9101', '45114', 0, 0],
 [33, '9111', '56141', 0, 0],
 [15, '9101', '1151', 0, 0],
 [18, '9101', '54136', 0, 0],
 [41, '9111', '81293', 0, 0]]

In [148]:
missing_df = pd.DataFrame.from_records(missing_data)
missing_df.columns = [STATE_KEY, LEGAL_FORM, BUSINESS_TYPE, ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS]
missing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10469 entries, 0 to 10468
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state_key             10469 non-null  int64 
 1   legal_form            10469 non-null  object
 2   business_type         10469 non-null  object
 3   establishment_count   10469 non-null  int64 
 4   revenue_in_thousands  10469 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 409.1+ KB


In [149]:
business_df = pd.concat([business_df, missing_df], ignore_index=True)
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119850 entries, 0 to 119849
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   state_key             119850 non-null  int64 
 1   legal_form            119850 non-null  object
 2   business_type         119850 non-null  object
 3   establishment_count   119850 non-null  int64 
 4   revenue_in_thousands  119850 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 4.6+ MB


## Holes have been filled

In [150]:
df = df.merge(business_df, on=STATE_KEY)
df.drop(columns=[STATE_KEY], inplace=True)
df = df.reindex([CAUSE, STATE_NAME, POPULATION, LEGAL_FORM, BUSINESS_TYPE, ESTABLISHMENT_COUNT, REVENUE_IN_THOUSANDS, DEATH_RATE], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1318350 entries, 0 to 1318349
Data columns (total 8 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   cause_name            1318350 non-null  object 
 1   state_name            1318350 non-null  object 
 2   population            1318350 non-null  int64  
 3   legal_form            1318350 non-null  object 
 4   business_type         1318350 non-null  object 
 5   establishment_count   1318350 non-null  int64  
 6   revenue_in_thousands  1318350 non-null  int64  
 7   death_rate            1318350 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 80.5+ MB


## Categorization
Let's make population, establishment_count, revenue_in_thousands and death_rate categories

Distribution plots

In [155]:
SMALL = "small"
MEDIUM = "medium"
LARGE = "large"
XLARGE = "x-large"

In [154]:
pd.qcut(df[POPULATION], 4, labels=[SMALL, MEDIUM, LARGE, XLARGE])

0          large
1          large
2          large
3          large
4          large
           ...  
1318345    small
1318346    small
1318347    small
1318348    small
1318349    small
Name: population, Length: 1318350, dtype: category
Categories (4, object): ['small' < 'medium' < 'large' < 'x-large']

In [163]:
pd.qcut(df[ESTABLISHMENT_COUNT], 10, duplicates="drop")

0          (1827.0, 3374050.0]
1          (1827.0, 3374050.0]
2          (1827.0, 3374050.0]
3          (1827.0, 3374050.0]
4          (1827.0, 3374050.0]
                  ...         
1318345          (-0.001, 3.0]
1318346          (-0.001, 3.0]
1318347          (-0.001, 3.0]
1318348          (-0.001, 3.0]
1318349          (-0.001, 3.0]
Name: establishment_count, Length: 1318350, dtype: category
Categories (9, interval[float64, right]): [(-0.001, 3.0] < (3.0, 7.0] < (7.0, 16.0] < (16.0, 34.0] ... (71.0, 165.0] < (165.0, 438.0] < (438.0, 1827.0] < (1827.0, 3374050.0]]

In [160]:
df[df[ESTABLISHMENT_COUNT] == 3374050]

,cause_name,state_name,population,legal_form,business_type,establishment_count,revenue_in_thousands,death_rate
103401,All causes,California,39512223,001,00,3374050,181655457,626.4250
105751,Alzheimer's disease,California,39512223,001,00,3374050,181655457,32.5750
108101,CLRD,California,39512223,001,00,3374050,181655457,34.1750
110451,Cancer,California,39512223,001,00,3374050,181655457,146.2750
112801,Diabetes,California,39512223,001,00,3374050,181655457,20.8500
115151,Heart disease,California,39512223,001,00,3374050,181655457,149.9750
117501,Influenza and pneumonia,California,39512223,001,00,3374050,181655457,15.3625
119851,Kidney disease,California,39512223,001,00,3374050,181655457,7.9875
122201,Stroke,California,39512223,001,00,3374050,181655457,36.1750
124551,Suicide,California,39512223,001,00,3374050,181655457,10.3375


In [152]:
frames_by_cause_dict = dict()
for cause_name in df[CAUSE].unique():
    mask = (df[CAUSE] == cause_name)
    new_df = df[mask].copy()
    new_df.drop(columns=[CAUSE], inplace=True)
    frames_by_cause_dict[cause_name] = new_df
frames_by_cause_dict["Heart disease"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 119850 entries, 11750 to 1306599
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   state_name            119850 non-null  object 
 1   population            119850 non-null  int64  
 2   legal_form            119850 non-null  object 
 3   business_type         119850 non-null  object 
 4   establishment_count   119850 non-null  int64  
 5   revenue_in_thousands  119850 non-null  int64  
 6   death_rate            119850 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 7.3+ MB
